In [0]:
!git clone https://github.com/deltat1995/AN2DL_Project1

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import os 
import numpy as np


In [0]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

In [0]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
%tensorflow_version 2.x
import os
import tensorflow as tf
import numpy as np

SEED = 345
tf.random.set_seed(SEED)  

cwd = os.getcwd()
cwd = os.path.join(cwd,'AN2DL_Project1')
cwd


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        brightness_range=(0.5,1.),
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant', 
                                        cval=0,
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255)

valid_data_gen = ImageDataGenerator(rescale=1./255)

In [0]:
dataset_dir = os.path.join(cwd, 'newDataset_03')

bs = 5

img_h=256
img_w=256

num_classes= 20

classes = [ 'owl',    # 0
            'galaxy', # 1
            'lightning', # 2
            'wine-bottle', # 3
            't-shirt', # 4
            'waterfall', # 5
            'sword', # 6
            'school-bus', # 7
            'calculator', # 8
            'sheet-music', # 9
            'airplanes', # 10
            'lightbulb', # 11
            'skyscraper', # 12
            'mountain-bike', # 13
            'fireworks', # 14
            'computer-monitor', # 15
            'bear', # 16
            'grand-piano', # 17
            'kangaroo', # 18
            'laptop']       # 19


# Training
training_dir = os.path.join(dataset_dir, 'training')
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               target_size=(img_h,img_w),
                                               batch_size=bs, 
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)  # targets are directly converted into one-hot vectors
validation_dir = os.path.join(dataset_dir, 'validation')
valid_gen = valid_data_gen.flow_from_directory(validation_dir,
                                               target_size=(img_h,img_w),
                                               batch_size=bs, 
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)  # targets are directly converted into one-hot vectors


class_indices = train_gen.class_indices
class_indices

TRANSFER LEARNING USING RESNET50 

In [0]:
#Resnet_50 = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', input_shape=(img_h, img_w,3))

In [0]:
vgg = tf.keras.applications.VGG16(weights='imagenet', include_top = False, input_shape = (img_h, img_w, 3))

In [0]:
vgg.summary()
#Resnet_50.summary()

In [0]:
finetuning = True

if finetuning:
    freeze_until = 15
     # layer from which we want to fine-tune
    
    for layer in vgg.layers[:freeze_until]:
        layer.trainable = False
else:
    vgg.trainable = False

model = tf.keras.Sequential()
model.add(vgg)
#model.add(tf.keras.layers.MaxPool2D(pool_size=(4, 4)))
#model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), padding='same'))
#model.add(tf.keras.layers.ReLU())
#model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=128, activation='relu')) 
model.add(tf.keras.layers.Dropout(rate=0.3)) #validation accuracy is higher than traning, might need to change this 
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))    

_ = tf.keras.utils.plot_model(model,'model_plot.png', show_shapes=True, show_layer_names=True,expand_nested=False)

model.summary()




In [0]:
# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate 
lr = 1e-6 #make the learning rate smaller as we do not want the model to change quickly 

optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# Validation metrics
metrics = ['accuracy']

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [0]:
import os
from datetime import datetime
import shutil
cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'classification_experiments')
shutil.rmtree(exps_dir, ignore_errors=True)
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'),
                                                   save_weights_only=False,) 
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)


history = model.fit_generator(train_gen,
                            validation_data = valid_gen,
                            callbacks=callbacks, 
                            epochs = 100)

In [0]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [0]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [0]:
from datetime import datetime

def create_csv(results, results_dir='./output/'):
    if not os.path.exists(results_dir):
      os.makedirs(results_dir)
    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [0]:
import os
from PIL import Image
import numpy as np

image_filenames = next(os.walk('./AN2DL_Project1/startDataset/test'))[2]                          
results = {}

for image_name in image_filenames:
    img = Image.open('./AN2DL_Project1/startDataset/test/'+image_name).convert('RGB')
    img = img.resize((img_w,img_h))
    img_array = np.array(img)
    img_array = img_array*1./255 #normalization
    img_array = np.expand_dims(img_array,0) #needed for fixed dim of input in the model
    output = model.predict(img_array)
    prediction = np.where(output == np.max(output))[1]  
    print('Image: '+str(image_name)+ ' Class: '+ list(class_indices.keys())[int(prediction)])
    results[image_name] = int(prediction)
    
create_csv(results)